!pip install elasticsearch==7.16.3

!pip install pandasql

In [2]:
import elasticsearch

from datetime import datetime
import pandas as pd
from pandasql import sqldf

In [3]:
with open('sophia-key.txt') as f:
    key = f.readlines()[0]

IP = "search.sophia2.org"
PORT = 9200
USER= "elastic_reader"
PASS= key

In [4]:
MEDIA_OUTLETS=["rioenlinea","diariopaillaco","diariomafil",
               "diariolaguino","diariolaunion","diariofutrono",
              "diariolagoranco","elnavegante","lapailapaillaco",
              "centralnoticiaspanguipulli","diariodevaldivia",
              "lavozdepaillaco","diariolanco","suractual",
              "noticiaslosrios","diarioriobueno","diariosanjose","diariolaguino",
              "diariocorral"]
COUNTRY = "chile"
KEYWORD = ""
SIMPLE_KEYWORD = True

In [5]:
def sophia_search(from_date,to_date,df_list):

    es = elasticsearch.Elasticsearch(
        IP,
        #port=PORT,
        http_auth=(USER, PASS)
    )

    match=""
    if (SIMPLE_KEYWORD):
        match="match"
    else:
        match="match_phrase"

    if len(KEYWORD)==0:
        query = { 
        "bool": { 
        "filter": [
            {"range": {
          "date": {
            "gte": from_date,
            "lt": to_date
          }}},
            { "term":  { "country": COUNTRY }},
            { "terms":  { "media_outlet": MEDIA_OUTLETS }} 
        ]
        }  
    }
    
    else:
        query = { 
        "bool": { 
        "must": [
            {match: { "text":KEYWORD}}
        ],
        "filter": [
            {"range": {
          "date": {
            "gte": from_date,
            "lt": to_date
          }}},
            { "term":  { "country": COUNTRY }},
            { "terms":  { "media_outlet": MEDIA_OUTLETS }} 
        ]
        }  
    }
        
    res = es.search(index="news", query=query, size=10000)
    print("Fechas: " + from_date + "|" + to_date)
    print("Son %d noticias encontradas..." % res['hits']['total']['value'])
    
    data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}

    df = pd.DataFrame(data)  

    for hit in res['hits']['hits']:
        id_news = hit['_source']['id_news']
        country = hit['_source']['country']
        media_outlet = hit['_source']['media_outlet']
        url = hit['_source']['url']
        title = hit['_source']['title']
        text = hit['_source']['text']
        date = hit['_source']['date']

        new_row = {'id_news':id_news, 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df_list.append(df)

In [6]:
df_list = []
months = [
    ["2022-01-01", "2022-01-31"],
    ["2022-02-01", "2022-02-28"],
    ["2022-03-01", "2022-03-31"],
    ["2022-04-01", "2022-04-30"],
    ["2022-05-01", "2022-05-31"],
    ["2022-06-01", "2022-06-30"],
    ["2022-07-01", "2022-07-31"],
    ["2022-08-01", "2022-08-31"],
    ["2022-09-01", "2022-09-30"],
    ["2022-10-01", "2022-10-31"],
    ["2022-11-01", "2022-11-30"],
    ["2022-12-01", "2022-12-31"]
]
for month in months:
    sophia_search(month[0],month[1],df_list)

Fechas: 2022-01-01|2022-01-31
Son 1096 noticias encontradas...
Fechas: 2022-02-01|2022-02-28
Son 1111 noticias encontradas...
Fechas: 2022-03-01|2022-03-31
Son 1350 noticias encontradas...
Fechas: 2022-04-01|2022-04-30
Son 1442 noticias encontradas...
Fechas: 2022-05-01|2022-05-31
Son 1559 noticias encontradas...
Fechas: 2022-06-01|2022-06-30
Son 1305 noticias encontradas...
Fechas: 2022-07-01|2022-07-31
Son 1322 noticias encontradas...
Fechas: 2022-08-01|2022-08-31
Son 1398 noticias encontradas...
Fechas: 2022-09-01|2022-09-30
Son 2304 noticias encontradas...
Fechas: 2022-10-01|2022-10-31
Son 1197 noticias encontradas...
Fechas: 2022-11-01|2022-11-30
Son 1069 noticias encontradas...
Fechas: 2022-12-01|2022-12-31
Son 1386 noticias encontradas...


In [7]:
#agrupar los dataframes mensuales en uno solo
df_anual = pd.concat(df_list)

#Eliminar noticias duplicadas que tienen la misma URL
df_anual = df_anual.drop_duplicates(subset='url', keep='first')

# Eliminar filas con valores NA o NaN en la columna 'text'
df_anual = df_anual.dropna(subset=['text'])

In [8]:
print(len(df_anual))
df_anual.head()

15147


,id_news,country,media_outlet,url,title,text,date
0,47314428.0,chile,suractual,https://www.suractual.cl/2022/01/24/ante-proye...,Ante proyecto Tierras Raras municipio y Unión ...,“Considerando la evolución que ha tenido el Pr...,2022-01-24
1,47719086.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/25/trac...,Tractocamión volcó esta mañana en puente Lloll...,El accidente se produjo pasadas las 06:00 AM y...,2022-01-25
2,47719110.0,chile,noticiaslosrios,https://www.noticiaslosrios.cl/2022/01/27/gobe...,Gobernadores Regionales de Los Ríos y Los Lago...,La instancia buscó generar un trabajo colabora...,2022-01-27
3,47700339.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Alegan inocencia de acusadas por cruel asesina...,La defensa de las dos mujeres acusadas de enca...,2022-01-05
4,47700425.0,chile,diariolaguino,https://www.diariolaguino.cl/noticia/actualida...,Destacan ejecución del Programa Vínculos en Lo...,Para dar protección y acompañamiento a adultos...,2022-01-03


In [9]:
#guardar en un CSV
df_anual.to_csv("data/noticias-losrios.csv")